In [13]:
import pickle

import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import mlflow
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline

import uuid
from azure.storage.blob import BlobServiceClient
import pickle

load_dotenv()


True

In [14]:
year = 2021
month = 3
taxi_type = 'green'

input_file = f'../../data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'


RUN_ID = os.getenv('RUN_ID', '36d1e16891874e2092cc2b1d74ede494')

In [15]:
# Get connection string from environment
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
if not connection_string:
    raise ValueError("AZURE_STORAGE_CONNECTION_STRING not found in environment variables")

os.environ["AZURE_STORAGE_CONNECTION_STRING"] = connection_string

# Connect to MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("green-taxi-duration-azure")

print("✅ MLflow with Azure Blob Storage ready!")

✅ MLflow with Azure Blob Storage ready!


In [16]:

#mlflow.set_tracking_uri("http://127.0.0.1:5001")
#mlflow.set_experiment("green-taxi-duration-azure")

#mlflow server \
#  --backend-store-uri=sqlite:///mlflow.db \
#  --default-artifact-root=wasbs://mlflow-artifacts@lifebonderblobml.blob.core.windows.net/ \
#  --host 0.0.0.0 \
#  --port 5001

#RUN_ID = 'c2b262c68c094ee68d9128dd0f272f75'
#model = mlflow.sklearn.load_model(f"runs:/{RUN_ID}/random_forest_taxi_duration_model")


In [17]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df['ride_id'] = generate_uuids(len(df))

    return df

def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts


In [18]:
def load_model(run_id):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    container_name = "mlflow-artifacts"
    blob_path = f"5/models/m-{run_id}/artifacts/model.pkl"

    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_path)
    model_data = blob_client.download_blob().readall()
    model = pickle.loads(model_data)
    return model


def apply_model(input_file, run_id, output_file):

    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)


    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id

    df_result.to_parquet(output_file, index=False)

In [19]:
os.makedirs(os.path.dirname(output_file), exist_ok=True)

apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

In [20]:
!ls output/green/

2021-02.parquet 2021-03.parquet


In [ ]:
#❯ jupyter nbconvert --to script score.ipynb